In [1]:
#Import the libraries that we are going to use
import pandas as pd
import time
#Import shapely for polygons visualization
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [2]:
#Read geographic data as pandas dataframe
geographics = pd.read_csv('geographic.csv')

#Initialize nta dictionary
nta_dictionary = {}
for nta in geographics.columns:
    nta_dictionary[nta] = ''

In [3]:
#Create a dictionary for each NTA with longitude and latitude as values
for nta in nta_dictionary:
    latitude = []
    longitude = [] 
    for i in geographics.loc[:,nta].values:
        if i < 0:
            latitude.append(i)
        elif i > 0:
            longitude.append(i)
    nta_dictionary[nta] = [longitude,latitude]

In [4]:
#Create a dictionary with nta as key and its polygon as value
nta_dictionary_lat_lon = {}
for nta in nta_dictionary.keys():
    polygon = []
    for lat,lon in zip(nta_dictionary[nta][0],nta_dictionary[nta][1]):
        lat_lon_tuple = (lat,lon)
        polygon.append(lat_lon_tuple)
        nta_dictionary_lat_lon[nta] = polygon

In [5]:
#Read Uber trips 2014 from csv file
uber_trips_2014 = pd.read_csv('uber_trips_2014.csv')

In [6]:
#Define a range to process the dataframe as chunks 
cut_range = [0,100]

In [7]:
#Cut original dataframe with the cut_range range
df_wip = uber_trips_2014.loc[cut_range[0]:cut_range[1]]

In [8]:
%%time
#Map every pickup point in every polygon to know where it fits
nta_serie = []
idx_dic = {}
#uber_trips_2014.shape[0]//10
for i in range(cut_range[0],cut_range[1]):
    point = Point(df_wip['pickup_latitude'][i],df_wip['pickup_longitude'][i])
    for j in nta_dictionary_lat_lon.keys():
        polygon = Polygon(nta_dictionary_lat_lon[j])
        if polygon.contains(point):
            idx_dic[i] = j
            break
        else:
            pass

CPU times: user 568 ms, sys: 92 µs, total: 569 ms
Wall time: 570 ms


In [9]:
#Create deep copy of Uber dataframe
df_test = df_wip.copy()

In [10]:
#Add nta column
df_test['Nta'] = pd.Series(idx_dic)

In [11]:
#Verify table integrity
df_test

,pickup_datetime,pickup_latitude,pickup_longitude,base,Nta
0,4/1/14 0:11,40.7690,-73.9549,B02512,NaN
1,4/1/14 0:17,40.7267,-74.0345,B02512,NaN
2,4/1/14 0:21,40.7316,-73.9873,B02512,MN22
3,4/1/14 0:28,40.7588,-73.9776,B02512,MN17
4,4/1/14 0:33,40.7594,-73.9722,B02512,MN19
...,...,...,...,...,...
96,4/1/14 6:56,40.7644,-73.9769,B02512,MN17
97,4/1/14 6:57,40.7726,-73.9532,B02512,NaN
98,4/1/14 6:59,40.7898,-73.9661,B02512,MN12
99,4/1/14 7:00,40.7839,-73.9703,B02512,NaN


In [14]:
#Save datframe with nta labels
df_test.to_csv('Uber_2014_nta' + '_' + str(cut_range[0]) + '_' + str(cut_range[1]) + '.csv')